# Iterating through the list of authors

In [1]:
import pickle
import re
import pandas as pd
from bs4 import BeautifulSoup
from parsing_page import parsing_author_page, parsing_article_page
import time
import requests
from os.path import exists


In [2]:
filename = "../data/mathnet_iam_authors_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

# Main functions for 

In [3]:
def need_to_scrap(author):
    # interesting_list = ["Поляков", "Бондаренко"]
    interesting_list = ["Бондаренко"]
    for person in interesting_list:
        if person in author['name']:
            return True
    return False
    

In [4]:
def get_author_info(mnid):
    #need to check if we already read it
    if mnid == "113970":
        filename = "../data/mn_author_page.pkl"
        with open(filename,'rb') as inp:
            author_page = pickle.load(inp)
    else:
        print(f"we need to request page for author mnid = {mnid}")
        # page_link = f"http://www.mathnet.ru/php/person.phtml?option_lang=rus&personid={mnid}"
        # 
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        # headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # # page_link ="http://mi.mathnet.ru/+ {}"
        # time.sleep(0.7)
        # response = requests.get(page_link, headers=headers, timeout=None)
        # if response == 200:
        #     author_page = response.content
        # author_page
        return None
    
    soup = BeautifulSoup(author_page, 'html.parser')
    # TODO:
    # Need to check if this is real page and not for bot response
    return parsing_author_page(soup) 

In [5]:
def get_pub_info(mnlink, status="testing"):
    print("../data/" + mnlink + ".pkl")
    print(exists("../data/" + mnlink + ".pkl"))
    if exists("../data/" + mnlink + ".pkl"):
        print("We read from file")
        filename = "../data/" + mnlink + ".pkl"
        with open(filename,'rb') as inp:
            pub_page = pickle.load(inp)
    else:
        
    # if mnlink == "vyurv213":
    #     filename = "../data/one_article_example.pkl"
    #     with open(filename,'rb') as inp:
    #         pub_page = pickle.load(inp)
    # else:
        print(f"we need to request page for pub_page mnlink = {mnlink}")
        page_link = "http://mi.mathnet.ru/" + mnlink
        print(page_link)
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # 
        time.sleep(0.5)
        response = requests.get(page_link, headers=headers, timeout=None)
        # response = 403
        print(response)
        print(type(response))
        if response.status_code == 200:
            pub_page = response.content
            # if not exists("../data/" + au_item['mn_link'] + ".pkl"):
            print("we save file: " + mnlink)
            save_html(pub_page,mnlink)
        else:
            print(response.status_code)
            print(type(response.status_code))
            print("response.status_code != 200:")
            return None
        # pub_page        
    soup = BeautifulSoup(pub_page, 'html.parser')
    if status == "testing":
        return parsing_article_page(soup)
    
    return parsing_article_page(soup)

In [6]:
def save_html(data, name):
    filename = "../data/" + name + ".pkl"
    with open(filename,'wb') as outp:
        pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [7]:
from data_types import Author, AuthorsDB
from data_types import Publication, PublicationsDB
from data_types import Abstract, AbstractsDB
audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()

In [8]:
print(pubdb)
print(absdb)
print(type(pubdb))
print(type(absdb))

<class 'data_types.PublicationsDB'>
<class 'data_types.AbstractsDB'>


In [9]:
audb.show()

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}


In [10]:
# '113970' in audb
print(type(audb))
audb.check_key('113970')

<class 'data_types.AuthorsDB'>


True

In [11]:
for indx, item in authors_dict.items():     
    if audb.check_key(item['mn_id']):
        print(indx)
        print(item)
    

87
{'name': 'Бондаренко Алексей Алексеевич', 'mn_id': '113970'}


In [12]:
for indx, item in authors_dict.items(): 
    
    if need_to_scrap(item):                
        new_author = Author(item["mn_id"])
        
        print(item)
        # get through pubs list on author page
        author_pubs = get_author_info(item["mn_id"])
        for pubs_indx, pubs_item in author_pubs.items():
            print(pubs_indx, pubs_item)            
            
            if pubs_item["mn_link"] is not None:                                
                pub_page_info = get_pub_info(pubs_item["mn_link"])
                new_pub = Publication(pubs_item["mn_link"])
                new_abs = Abstract(pubs_item["mn_link"])
                
                if pub_page_info is not None:
                    new_author.update_author_info(pubs_item, pub_page_info)
                    new_pub.update_publication_info(pubs_item, pub_page_info)
                    new_abs.update_abstract_info(pubs_item, pub_page_info)
                    pubdb.update_data(new_pub.convert2dict())                            
                    absdb.update_data(new_abs.convert2dict())        

                    # update_pub_db(pubs_item,pub_page_info )
                    # update_abstract_db(pubs_item,pub_page_info )
                    for page_indx, page_item  in pub_page_info.items():
                        print(page_indx)
                        print(page_item)                      
        # new_author.show()
        print(new_author.convert2dict())
        audb.update_data(new_author.convert2dict())
        audb.show()
        pubdb.show()
        absdb.show()
audb.save()
pubdb.save()
absdb.save()

{'name': 'Бондаренко Алексей Алексеевич', 'mn_id': '113970'}
1 {'name': 'А.А.Бондаренко, П.А.Ляхов, М.В.Якобовский, “Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 8:2 (2019), 76–91', 'mn_link': 'vyurv213', 'elib_id': '38073495'}
../data/vyurv213.pkl
True
We read from file
id vyurv213 in self.db
author_names
['А.\xa0А.\xa0Бондаренко', 'П.\xa0А.\xa0Ляхов', 'М.\xa0В.\xa0Якобовский']
author_id
['113970', '148811', '22428']
abstract
Увеличивающийся рост числа компонент суперкомпьютеров приводит специалистов в области HPC к неблагоприятным оценкам для будущих суперкомпьютеров: диапазон среднего времени между отказами будет составлять от 1 часа до 9 часов. Данная оценка ставит под вопрос возможность проведения длительных расчетов на суперкомпьютерах. В работе предлагается метод восстановления после отказов, не требующий возврата большинства процессов к последней контрольной точке, что мо

# Testing AuthorsDB functions

In [13]:
bond = {'007':
    {'links': ['vyurv213', 'vyurv1', 'vyurv46'],
     'years': ['2019', '2015', '2014'],
     'coathors': {'148811', '22428'}}}
bond2 = {'007':
    {'links': ['vyurv213', 'vyurv1', 'vyurv46','cool_paper'],
     'years': ['2019', '2015', '2014','cool_year'],
     'coathors': {'148811', '22428','009'}}}
bond3 = {'007':
    {'links': ['mit'],
     'years': ['1955'],
     'coathors': {'005'}}}

In [14]:
audb.show()
audb.update_data(bond)
audb.show()
audb.update_data(bond2)
audb.show()
audb.update_data(bond3)
audb.show()

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
id 007 in self.db.keys()
printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
i

In [15]:
bond2 = {'008':
    {'links': ['vyurv213', 'vyurv1', 'vyurv46'],
     'years': ['2019', None, '2014'],
     'coathors': {'148811', '22428'}}}
bond2_new_inf = {'008':
    {'links': ['mit'],
     'years': [None],
     'coathors': {'005'}}}

In [16]:
audb.show()
audb.update_data(bond2)
audb.show()
audb.update_data(bond2_new_inf)
audb.show()

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
id 008 in self.db.keys()
printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
i

In [17]:
audb.save()

In [18]:
new_db = AuthorsDB()
new_db.load()
new_db.show()

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}


# Testing AbstractsDB

In [19]:
audb.show()
pubdb.show()
absdb.show()

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
printing PublicationsDB:
ind = vyurv213
item = {'author_id': ['113970', '148811', '22428'], 'doi': '10.14529/cmse190205', 'udk': ['004.052.3'], 'send': ['20.11.2018'], 'type': 'Статья', 'reference': 'А.А.Бондаренко, П.А.Ляхов, М.В.Якобовский, “Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 8:2 (2019), 76–91'}
ind = vyurv1
item = {'author_id': ['113970', '22428'], 'doi': '10.14529/cmse150301', 'udk': ['00

In [20]:
audb.show()
audb.pop_key('113970')
audb.show()
audb.pop_key('113970')

printing AuthorsDB:
ind = 113970
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'cool_paper'], 'years': ['2019', '2015', '2014', 'cool_year'], 'coathors': {'22428', '148811', '009'}}
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
printing AuthorsDB:
ind = 007
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', '2015', '2014', '1955'], 'coathors': {'22428', '148811', '005'}}
ind = 008
item = {'links': ['vyurv213', 'vyurv1', 'vyurv46', 'mit'], 'years': ['2019', None, '2014', None], 'coathors': {'22428', '148811', '005'}}
There is no such element in AuthorsDB as 113970


In [21]:
pubdb.show()
pubdb.pop_key('vyurv213')
pubdb.show()
pubdb.pop_key('vyurv213')

printing PublicationsDB:
ind = vyurv213
item = {'author_id': ['113970', '148811', '22428'], 'doi': '10.14529/cmse190205', 'udk': ['004.052.3'], 'send': ['20.11.2018'], 'type': 'Статья', 'reference': 'А.А.Бондаренко, П.А.Ляхов, М.В.Якобовский, “Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 8:2 (2019), 76–91'}
ind = vyurv1
item = {'author_id': ['113970', '22428'], 'doi': '10.14529/cmse150301', 'udk': ['004.052.3'], 'send': ['13.04.2015'], 'type': 'Статья', 'reference': 'А.А.Бондаренко, М.В.Якобовский, “Моделирование отказов в высокопроизводительных вычислительных системах в рамках стандарта MPI и его расширения ULFM”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 4:3 (2015), 5–12'}
ind = vyurv46
item = {'author_id': ['113970', '22428'], 'doi': None, 'udk': ['004.052.3'], 'send': ['05.08.2014'], 'type': 'Статья', 'reference': 'А.А.Бондаренко, М.В.Якобовский, “Обеспечение отказоустойчивости

In [22]:
absdb.show()
absdb.pop_key('vyurv213')
absdb.show()
absdb.pop_key('vyurv213')

printing AbstractsDB:
ind =
vyurv213
row =
abstract    Увеличивающийся рост числа компонент суперкомп...
keywords    расширение ULFM, контрольные точки, координиро...
Name: vyurv213, dtype: object
ind =
vyurv1
row =
abstract    Рассматривается проблема выполнения длительных...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv1, dtype: object
ind =
vyurv46
row =
abstract    Рассматриваются вопросы, связанные с проведени...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv46, dtype: object
printing AbstractsDB:
ind =
vyurv1
row =
abstract    Рассматривается проблема выполнения длительных...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv1, dtype: object
ind =
vyurv46
row =
abstract    Рассматриваются вопросы, связанные с проведени...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv46, dtype: object
There is no such element in AbstractsDB as vyurv213


In [23]:
a = ["asd","asd",["asd","asd"]]
if None in a:
    print("None in a")
else:
    print("None not in a")

None not in a


In [24]:
myd = {1:"a",
       2:"b",
       3:"c",
       4: None,
       5: ""}
print(myd.values())
if "" in myd.values():       
    print(" '' in myd.values()")
for val in myd.values():
       print(val)


dict_values(['a', 'b', 'c', None, ''])
 '' in myd.values()
a
b
c
None



In [25]:
myd.pop(2)
print(myd)

{1: 'a', 3: 'c', 4: None, 5: ''}


In [26]:
myd.update({5:{'a':{"b":91}}})

In [27]:
print(myd)

{1: 'a', 3: 'c', 4: None, 5: {'a': {'b': 91}}}


In [28]:
myd.pop(5)

{'a': {'b': 91}}

In [29]:
print(myd)

{1: 'a', 3: 'c', 4: None}
